In [30]:
from __future__ import print_function
import os
import sys
import re
import time
from PIL import Image
import numpy as np
import pandas as pd
import theano as th
# from matplotlib import pyplot as plt
# %matplotlib inline

# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import confusion_matrix
# from CSE6240_utilities import plot_confusion_matrix
# import matplotlib.pylab as pylab

# from scipy.stats import pearsonr

In [31]:
class image_parser(object):
    
    def __init__(self,datapath=None,folder=None,images=None,image_names=None,
                 min_rows=100,min_cols=100,verbose=False):
        self.datapath=datapath
        self.folder=folder
        self.images=images
        self.image_names=image_names
        self.min_rows=min_rows
        self.min_cols=min_cols
        self.verbose=verbose

    def read(self):
        self.images=np.empty(self.min_rows*self.min_cols)
        self.image_names=[]
        for base, dirs, files in os.walk (self.datapath+'/'+self.folder+'/'):
            for filename in files:
                if self.verbose: print("reading..."
                                  +self.datapath+'/'+self.folder+'/'+filename)
                name_JPEG = re.match (r'^(.*)\.JPEG$',filename)
                if name_JPEG:
                    filepath = os.path.join (base, filename)
                    image = Image.open (filepath,'r'
                                       ).resize((self.min_rows,self.min_cols)).convert("L")
                    image = np.array(image).reshape(-1)
                    self.images=np.vstack((self.images,image))
                    self.image_names.append(filename)

In [25]:
datapath="/Users/jiajiechen/Desktop/project2-CSE6240/Data"

In [52]:
animals=image_parser(folder="01-Animal",datapath=datapath,
                     verbose=False,min_rows=64,min_cols=64)

In [53]:
animals.read()

In [55]:
animals.images.shape

(1572, 4096)

In [62]:
class k_Sparse_AutoEncoder(object):
    def __init__(self, X, hidden_size, activation_function,
                 output_function):
        #X is the data, an m x n numpy matrix
        assert type(X) is np.ndarray
        assert len(X.shape)==2
        self.X=X
        self.X=th.shared(name='X', value=np.asarray(self.X, 
                         dtype=th.config.floatX),borrow=True)
        #The config.floatX and borrow=True stuff is to get this to run
        #fast on the gpu.
        self.n = X.shape[1]
        self.m = X.shape[0]
        #Hidden_size is the number of neurons in the hidden layer, an int.
        assert type(hidden_size) is int
        assert hidden_size > 0
        self.hidden_size = hidden_size
        initial_W = np.asarray(np.random.uniform(
                 low = -4 * np.sqrt(6. / (self.hidden_size + self.n)),
                 high = 4 * np.sqrt(6. / (self.hidden_size + self.n)),
                 size = (self.n, self.hidden_size)), dtype=th.config.floatX)
        self.W = th.shared(value=initial_W, name='W', borrow=True)
        self.b1 = th.shared(name='b1', value=np.zeros(shape=(self.hidden_size,),
                            dtype=th.config.floatX),borrow=True)
        self.b2 = th.shared(name='b2', value=np.zeros(shape=(self.n,),
                            dtype=th.config.floatX),borrow=True)
        self.activation_function=activation_function
        self.output_function=output_function
                     
    def train(self, n_epochs = 100, mini_batch_size=1, learning_rate=0.1):
        index = th.tensor.lscalar()
        x = th.tensor.matrix('x')
        params = [self.W, self.b1, self.b2]
        hidden = self.activation_function(th.tensor.dot(x, self.W)+self.b1)
        output = th.tensor.dot(hidden,th.tensor.transpose(self.W))+self.b2
        output = self.output_function(output)
         
        #Use cross-entropy loss.
        L = -th.tensor.sum(x*th.tensor.log(output) + (1-x)*th.tensor.log(1-output), axis=1)
        cost=L.mean()       
        updates=[]
         
        #Return gradient with respect to W, b1, b2.
        gparams = th.tensor.grad(cost,params)
         
        #Create a list of 2 tuples for updates.
        for param, gparam in zip(params, gparams):
            updates.append((param, param-learning_rate*gparam))
         
        #Train given a mini-batch of the data.
        train = th.function(inputs=[index], outputs=[cost], updates=updates,
                            givens={x:self.X[index:index + mini_batch_size,:]})

        start_time = time.time()
        for epoch in xrange(n_epochs):
            print("Epoch:",epoch)
            for row in xrange(0,self.m, mini_batch_size):
                train(row)
        end_time = time.time()
        print("Average time per epoch=", (end_time-start_time)/n_epochs)
                    
    def get_hidden(self,data):
        x = th.tensor.dmatrix('x')
        hidden = self.activation_function(th.tensor.dot(x,self.W)+self.b1)
        transformed_data = th.function(inputs=[x], outputs=[hidden])
        return transformed_data(data)
     
    def get_weights(self):
        return [self.W.get_value(), self.b1.get_value(), self.b2.get_value()]

In [63]:
animals.images = animals.images / 255.0

In [66]:
help(activation_function)

Help on Elemwise in module theano.tensor.elemwise object:

class Elemwise(theano.gof.op.OpenMPOp)
 |  Generalizes a scalar op to tensors.
 |  
 |  All the inputs must have the same number of dimensions. When the
 |  Op is performed, for each dimension, each input's size for that
 |  dimension must be the same. As a special case, it can also be 1
 |  but only if the input's broadcastable flag is True for that
 |  dimension. In that case, the tensor is (virtually) replicated
 |  along that dimension to match the size of the others.
 |  
 |  The dtypes of the outputs mirror those of the scalar Op that is
 |  being generalized to tensors. In particular, if the calculations
 |  for an output are done inplace on an input, the output type must
 |  be the same as the corresponding input type (see the doc of
 |  scalar.ScalarOp to get help about controlling the output type)
 |  
 |  Parameters
 |  ----------
 |  scalar_op
 |      An instance of a subclass of scalar.ScalarOp which works uniquely

In [64]:
activation_function=th.tensor.nnet.sigmoid
output_function=activation_function
A = k_Sparse_AutoEncoder(animals.images,20,
                         activation_function,output_function)
A.train(n_epochs=2, mini_batch_size=20) # with 1000 hidden units for 20 epochs with minibatch size of 20
W = np.transpose(A.get_weights()[0])


print(A.get_hidden(animals.images)[0])

Epoch: 0
Epoch: 1
Average time per epoch= 0.406615972519
[[ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 ..., 
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]
 [ nan  nan  nan ...,  nan  nan  nan]]


In [ ]:
# # you may change the folders' name to your customization
# images = []
# image_names = []

# images, image_names = readImg("01-Animal", images, image_names, 0)
# images, image_names = readImg("02-Fungus", images, image_names, 1)
# images, image_names = readImg("03-Geological Formation", images, image_names, 2)
# images, image_names = readImg("04-Person", images, image_names, 3)
# images, image_names = readImg("05-Plant, flora, plant life", images, image_names, 4)
# images, image_names = readImg("06-Sport", images, image_names, 5)